Reference:

https://levelup.gitconnected.com/create-your-own-finite-volume-fluid-simulation-with-python-8f9eab0b8305


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def getConserved(rho,vx,vy,vol):
    '''
    Calculate the conserved variable from the primitive
    
    rho          is the matrix of cell densities
    vx           is the matrix of cell x-velocity
    vy           is the matrix of cell y-velocity
    vol          is cell volume

    Mass         is the matrix of cell mass
    Momx         is the matrix of cell x-momentum
    Momy         is the matrix of cell y-momentum 
    '''
    Mass = rho * vol
    Momx = rho * vx * vol
    Momy = rho * vy * vol

    return Mass,Momx,Momy
    
def getPrimitive(Mass,Momx,Momy,vol):
    
    rho = Mass/vol
    vx  = Momx / Mass
    vy  = Momy / Mass
    
    return rho,vx,vy

def getGradient(f,dx):
    '''
    f           is the matrix of a field
    dx          is the space cell size of field
    f_dx        is the matrix of derivative of f in the x-direction
    f_dy        is the matrix of derivative of f in the y-direction
    '''
    f_dx = (np.roll(f,-1,axis=0) - np.roll(f,1,axis=0)) / (2 * dx)
    f_dy = (np.roll(f,-1,axis=1) - np.roll(f,1,axis=1)) / (2 * dx)

    return f_dx,f_dy

def extrapolateSpaceToFace(f,f_dx,f_dy,dx):
    '''
    calculate the interFace value
    f         is the matrix of a field 
    f_dx      is the matrix of derivate in x 
    f_dy      is the matrix of derivate in y
    dx        is the cell size
    f_XL      is the matrix of value in 'Left' interfaces along x
    f_YL      is the matrix of value in 'Left' interfaces along y
    f_XR      is the matrix of value in 'Right' interfaces along x
    f_YR      is the matrix of value in 'Right' interfaces along y
    '''
    f_XL = f - f_dx * dx / 2
    f_XL = np.roll(f_XL,-1,axis=0)
    f_XR = f + f_dx * dx / 2

    f_YL = f - f_dy * dx / 2
    f_YL = np.roll(f_YL,-1,axis=1)
    f_YR = f + f_dy * dx / 2 

    return f_XL,f_XR,f_YL,f_YR

def getFlux()


def main():
    
    #control param
    N           = 100 # the number of cells
    t           = 0   # the current time
    t_end       = 100 # the end time of simulation
    CFL_number  = 0.5 # the coutant factor number

    #Mesh param
    Lbox        = 1.0 # a square
    dx          = Lbox / N
    vol         = dx**2*1
    x_c_list    = np.linspace(0.5*dx,Lbox-0.5*dx,N)
    X,Y         = np.meshgrid(x_c_list,x_c_list)#  x(y)crood at [i,j]
    
    print("===== ====== ====== ====== ===== ===== ===== ===== ===== ===== =====")
    print(X)
    print("===== ====== ====== ====== ===== ===== ===== ===== ===== ===== =====")
    print(Y)
    print("===== ====== ====== ====== ===== ===== ===== ===== ===== ===== =====")

    #initial Conditions
    rho = 0*X + 1
    vx  = 0.5 * np.sin(2*np.pi*Y)
    vy  = 0.5 * np.sin(2*np.pi*X)

    #Get the conserved variables
    Mass, Momx, Momy = getConserved(rho,vx,vy,vol)
    fig = plt.figure(figsize =(4,4),dpi = 400)
    cmap = plt.cm.bwr
    cmap.set_bad('LightGray')
    outputCount = 1

    #simulation loop
    while t < t_end:
        #get primitive variable
        rho,vx,vy = getPrimitive(Mass,Momx,Momy,vol)

        #get the time step
        dt = CFL_number * np.min(dx / (1 + np.sqrt(vx**2 + vy**2)))

        #judge the plot time
        IsPlotTime = False

        # calcute the gradients
        rho_dx,rho_dy = getGradient(rho,dx)
        vx_dx,vx_dy   = getGradient(vx,dx)
        vy_dx,vy_dy   = getGradient(vy,dx)
        P_dx = rho_dx
        P_dy = rho_dy
        # extrapolate half-step in time
        rho_prime = rho - 0.5*dt * ( vx * rho_dx + rho * vx_dx + vy * rho_dy + rho * vy_dy)
        vx_prime  = vx  - 0.5*dt * ( vx * vx_dx + vy * vx_dy + (1/rho) * P_dx )
        vy_prime  = vy  - 0.5*dt * ( vx * vy_dx + vy * vy_dy + (1/rho) * P_dy )

        #extrapolate in space to face centers
        rho_XL, rho_XR, rho_YL, rho_YR = extrapolateSpaceToFace(rho_prime,rho_dx,rho_dy,dx)
        vx_XL,  vx_XR,  vx_YL,  vx_YR  = extrapolateSpaceToFace(vx_prime,  vx_dx,  vx_dy,  dx)
        vy_XL,  vy_XR,  vy_YL,  vy_YR  = extrapolateSpaceToFace(vy_prime,  vy_dx,  vy_dy,  dx)

        #get the flux 
        flux_mass_x, flux_momx_x, flux_momy_x = getFlux(rho_XL,rho_XR,vx_XL,vx_XR,vy_XL,vy_XR)
        flux_Mass_Y, flux_Momy_Y, flux_Momx_Y = getFlux(rho_YL, rho_YR, vy_YL, vy_YR, vx_YL, vx_YR)

		
        
    


if __name__=="__main__":
    main()
